<a href="https://colab.research.google.com/github/alibh97/Sexism_Detection/blob/master/Project_NLP_task1_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**This task is gonna take a tweet and classify it as sexist or not sexist**

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 36.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd

# Load the uploaded file into a pandas dataframe
# data = pd.read_excel("modified_EXIST2021_training_test_labeled_EXIST2023_training_dev_task1.xlsx")
data = pd.read_excel("modified_EXIST2023_training_dev_all.xlsx")

# Display the first few rows of the dataframe to understand its structure
data.head()


,0,1,2,3,4
0,100001,"@TheChiflis Ignore the other, he's an asshole....",1,1.1,"[2, 3]"
1,100002,@ultimonomada_ If comicsgate is anything like ...,0,-,-
2,100003,"@Steven2897 Read about Gamergate, and how it h...",0,-,-
3,100004,"@Lunariita7 A rather unfortunate social delay,...",1,1,[0]
4,100005,@novadragon21 @icep4ck @TvDannyZ So since this...,1,2,"[0, 1, 2]"


In [ ]:
from transformers import BertTokenizer

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)


In [ ]:
import torch
import numpy as np


In [ ]:
# Tokenize all of the sentences and map the tokens to their word IDs
input_ids = []
attention_masks = []

for tweet in data[1]:
    encoded_dict = tokenizer.encode_plus(
        tweet,
        add_special_tokens=True,  # Add '[CLS]' and '[SEP]'
        max_length=128,            # Pad & truncate all sentences
        pad_to_max_length=True,
        return_attention_mask=True,  # Construct attention masks
        return_tensors='pt',         # Return PyTorch tensors
    )

    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
# Assuming your labels are stored as a NumPy array, you can convert them to int64:
labels = torch.tensor(data[2].values.astype(np.int64))

# labels = torch.tensor(data['1'].values)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset
dataset = TensorDataset(input_ids, attention_masks, labels)

# Create a 90-10 train-validation split
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])


In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# Create data loaders for training and validation sets
batch_size = 64

train_dataloader = DataLoader(
    train_dataset,
    sampler=RandomSampler(train_dataset),
    batch_size=batch_size
)

validation_dataloader = DataLoader(
    val_dataset,
    sampler=SequentialSampler(val_dataset),
    batch_size=batch_size
)


In [ ]:
from transformers import BertForSequenceClassification, AdamW

# Load BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=2,
    output_attentions=False,
    output_hidden_states=False
)

# Move the model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
import numpy as np

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)


In [ ]:
from transformers import get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score



epochs = 4

# Create a learning rate scheduler
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=len(train_dataloader) * epochs
)

# Training loop
for epoch in range(epochs):
    # Training
    model.train()
    total_loss = 0
    for batch in train_dataloader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Clear gradients
        model.zero_grad()

        # Forward pass
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        loss = outputs[0]
        total_loss += loss.item()

        # Backward pass
        loss.backward()

        # Update weights
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch + 1}, Training Loss: {avg_train_loss:.4f}")

    # Validation
    model.eval()
    eval_loss, eval_accuracy = 0, 0
    all_logits = []
    true_labels = []
    for batch in validation_dataloader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        with torch.no_grad():
            outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

        logits = outputs[0]
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        all_logits.extend(logits)
        true_labels.extend(label_ids)
        # Calculate accuracy for this batch
        eval_accuracy += flat_accuracy(logits, label_ids)

    avg_val_accuracy = eval_accuracy / len(validation_dataloader)
    print(f"Epoch {epoch + 1}, Validation Accuracy: {avg_val_accuracy:.4f}")
    # Convert logits to label predictions
    pred_labels = np.argmax(np.array(all_logits), axis=1)

    accuracy = accuracy_score(true_labels, pred_labels)
    precision = precision_score(true_labels, pred_labels)
    recall = recall_score(true_labels, pred_labels)
    f1 = f1_score(true_labels, pred_labels)

    print(f"Validation Accuracy: {accuracy:.4f}")
    print(f"Validation Precision: {precision:.4f}")
    print(f"Validation Recall: {recall:.4f}")
    print(f"Validation F1 Score: {f1:.4f}")


Epoch 1, Training Loss: 0.5481
Epoch 1, Validation Accuracy: 0.7804
Validation Accuracy: 0.7802
Validation Precision: 0.8028
Validation Recall: 0.6571
Validation F1 Score: 0.7227
Epoch 2, Training Loss: 0.4151
Epoch 2, Validation Accuracy: 0.7931
Validation Accuracy: 0.7902
Validation Precision: 0.7217
Validation Recall: 0.8444
Validation F1 Score: 0.7782
Epoch 3, Training Loss: 0.3346
Epoch 3, Validation Accuracy: 0.7855
Validation Accuracy: 0.7839
Validation Precision: 0.7119
Validation Recall: 0.8473
Validation F1 Score: 0.7737
Epoch 4, Training Loss: 0.2582
Epoch 4, Validation Accuracy: 0.7931
Validation Accuracy: 0.7902
Validation Precision: 0.7406
Validation Recall: 0.7983
Validation F1 Score: 0.7684


In [ ]:
def predict(sentence, model, tokenizer):
    # Step 1: Tokenize the sentence
    encoded_dict = tokenizer.encode_plus(
        sentence,
        add_special_tokens=True,
        max_length=64,
        pad_to_max_length=True,
        return_attention_mask=True,
        return_tensors='pt',
    )

    input_id = encoded_dict['input_ids'].to(device)
    attention_mask = encoded_dict['attention_mask'].to(device)

    # Step 2: Model Prediction
    with torch.no_grad():
        outputs = model(input_id, token_type_ids=None, attention_mask=attention_mask)

    logits = outputs[0]
    predicted_label = torch.argmax(logits, dim=1).item()

    # Step 3: Interpret the Prediction
    if predicted_label == 1:
        return "Sexist"
    else:
        return "Not Sexist"

# Example usage:
sentence = "your body is amazing."
prediction = predict(sentence, model, tokenizer)
print(f"The sentence is predicted as: {prediction}")


The sentence is predicted as: Not Sexist


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
